In [29]:
import matplotlib.pyplot as plt

def sectionGenerateRec(width, height, cover, ds, db, nx, ny, plot=True):
    """
    1) Compute rebar layout.
    2) Plot cross-section with rebar layout.
    3) Return results dictionary.
    """

    # -----------------------------------------------------------
    # 1) Outer rectangle corners (unconfined section)
    # -----------------------------------------------------------
    section_corners = [
        (0.0,  0.0),       
        (width,  0.0),       
        (width,  height),    
        (0.0,  height)
    ]

    # -----------------------------------------------------------
    # 2) Stirrup center-line corners (confined section)
    #    Offset from the outer face by (cover + 0.5*ds)
    # -----------------------------------------------------------
    stirrup_offset = cover + (0.5 * ds)
    stirrup_corners = [
        (stirrup_offset,             stirrup_offset),
        (width - stirrup_offset,     stirrup_offset),
        (width - stirrup_offset,     height - stirrup_offset),
        (stirrup_offset,             height - stirrup_offset)
    ]

    # -----------------------------------------------------------
    # 3) Main bar offset from the outer face
    #    = cover + ds + 0.5*db
    # -----------------------------------------------------------
    rebar_offset = cover + ds + 0.5 * db

    x_left  = rebar_offset
    x_right = width - rebar_offset
    y_bot   = rebar_offset
    y_top   = height - rebar_offset

    # -----------------------------------------------------------
    # 4) Compute rebar coordinates
    # -----------------------------------------------------------
    rebar_coords = []

    # Horizontal bars (top & bottom)
    if nx == 1:
        x_positions = [(x_left + x_right) / 2.0]
    else:
        dx = (x_right - x_left) / (nx - 1)
        x_positions = [x_left + i * dx for i in range(nx)]

    for x_pos in x_positions:
        rebar_coords.append((x_pos, y_bot))  # bottom
        rebar_coords.append((x_pos, y_top))  # top

    # Vertical bars (left & right)
    if ny == 1:
        y_positions = [(y_bot + y_top) / 2.0]
    else:
        dy = (y_top - y_bot) / (ny - 1)
        y_positions = [y_bot + i * dy for i in range(ny)]

    for y_pos in y_positions:
        rebar_coords.append((x_left,  y_pos))   # left
        rebar_coords.append((x_right, y_pos))   # right

    # Remove duplicates (corner bars appear in both sets)
    rebar_coords = list(set(rebar_coords))
    # Sort in ascending order of y, then x for consistent output
    rebar_coords.sort(key=lambda pt: (pt[1], pt[0]))

    # -----------------------------------------------------------
    # 5) Count total number of bars
    # -----------------------------------------------------------
    total_no_rsb = len(rebar_coords)

    # -----------------------------------------------------------
    # 6) Plot the layout
    # -----------------------------------------------------------
    fig, ax = plt.subplots(figsize=(5, 6))

    # Outer rectangle (unconfined section)
    x_vals = [p[0] for p in section_corners] + [section_corners[0][0]]
    y_vals = [p[1] for p in section_corners] + [section_corners[0][1]]
    ax.plot(x_vals, y_vals, 'k-', linewidth=2) # label="Unconfined Section"

    # Stirrup center-line rectangle
    sx_vals = [p[0] for p in stirrup_corners] + [stirrup_corners[0][0]]
    sy_vals = [p[1] for p in stirrup_corners] + [stirrup_corners[0][1]]
    ax.plot(sx_vals, sy_vals, 'b--', linewidth=1.5) #label="Stirrup (Center-Line)"

    # Rebar points
    rx = [p[0] for p in rebar_coords]
    ry = [p[1] for p in rebar_coords]
    ax.scatter(rx, ry, s=60, c='red', marker='o') #Main Bars"

    # if show_labels:
        # Label corners
    for i, pt in enumerate(section_corners, start=1):
        ax.text(pt[0], pt[1], f"C{i}", color='blue', fontsize=9, ha='left', va='bottom')
        # Label stirrup corners
    for i, pt in enumerate(stirrup_corners, start=1):
        ax.text(pt[0], pt[1], f"S{i}", color='green', fontsize=9, ha='left', va='bottom')
        # Label rebar
    for i, (xr, yr) in enumerate(rebar_coords, start=1):
        ax.text(xr, yr, f"R{i}", color='red', fontsize=8, ha='left', va='bottom')

    # Formatting
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-10, width + 10)
    ax.set_ylim(-10, height + 10)
    ax.set_xlabel("X-axis (mm)")
    ax.set_ylabel("Y-axis (mm)")
    ax.set_title("RC Cross Section")
    ax.legend()
    ax.grid(plot)

    # plt.show()

    # -----------------------------------------------------------
    # 7) Return dictionary with everything
    # -----------------------------------------------------------
    return {
        "section_corners":  section_corners,
        "stirrup_corners":  stirrup_corners,
        "rebar_coords":     rebar_coords,
        "total_no_rsb":     total_no_rsb,
        # "plot" : plt.show()
    }

# compute_and_plot_beam_rebar(200, 450, 40, 12, 16, 3, 3)

def generateFiber(name,section_assign,type="PO"):
    
    if type == "PO": 
        encode_type = "1"
    elif type == "TH":
        encode_type = "0"  
    fiber_mgt = f'''
    *FIBER-DIVISION    ; Section Division for Fiber Model
    NAME={name}, {section_assign}, {encode_type}
    '''
    print(fiber_mgt)
generateFiber("FIBER",11,type="PO")   

# NAME=NL test, 1, 0
#        C3000_1C1_CONF, C3000_1C1_CONF, C3000_1C1_CONF, C3000_1C1_CONF, C3000_1C1_CONF, C3000_1C1_CONF, YES, 0, 0, 0, 0, 0, 0, 0, 0
#        121, 198, 220, 245, 242, 31, 214, 126, 176, 63, 174, 179, 40, 130, 



    *FIBER-DIVISION    ; Section Division for Fiber Model
    NAME=FIBER, 11, 1
    
